# Report NoteBook

To do:  
  - Simple Tests - Garantee that everything is working.  
Construct a python test script  

Extras


Done:
  - Send the records as json
  - Add all verifications to the different actions  
  - Garantee the Records from different actions will go to different places.  

# Main file

In [31]:
%%file main.py
from flask import Flask, request, jsonify
import boto3
import json

# Create a Flask applicatiob
app = Flask(__name__)

#Creates a client to interact with firhose
client_firehose = boto3.client('firehose',region_name='us-east-2')


#Welcome Message
@app.route('/')
def begin():
    return ("Welcome, This API as 3 actions track,alias,profile")




####### /track  #####

@app.route('/track',methods = ['POST'])
def track():
    user_events = request.get_json()
    
    #Correct Json Format Verification
    if list(user_events.keys())==['userId', 'events']:
        
        #creates a string with the json format of the request
        user_events_json=json.dumps(user_events)
        
        #Sends the string to firehose Delivery Stream projectx-track which will write periodically on amazon S3 on the preffix track
        response = client_firehose.put_record(DeliveryStreamName='projectx-track',Record={'Data': user_events_json})  
        return(str(response['ResponseMetadata']['HTTPStatusCode']))
    return('Error: Wrong Json Format/fiels')
    




###### /alias #####

@app.route('/alias',methods = ['POST'])
def alias():
    alias_user = request.get_json()
    confirmation='false'
    
    #Correct Json Format Verification
    if list(alias_user.keys())==  ["newUserId","originalUserId","timestampUTC"]:
        
        #creates a string with the json format of the request
        alias_user_json=json.dumps(alias_user)
        
        #Sends the string to firehose Delivery Stream projectx-alias which will write periodically on amazon S3 on the preffix alias
        response = client_firehose.put_record(DeliveryStreamName='projectx-alias',Record={'Data': alias_user_json})
        return(str(response['ResponseMetadata']['HTTPStatusCode']))
    return('Error: Wrong Json Format/fiels')




###### /profile #####

@app.route('/profile',methods = ['POST'])
def profile():    
    user_profile = request.get_json()
    confirmation='false'
    
    #Correct Json Format Verification
    if list(user_profile.keys())==  ["userId","attributes","timestampUTC"]:
        
        #creates a string with the json format of the request
        alias_user_json=json.dumps(alias_user)
        
        #Sends the string to firehose Delivery Stream projectx-profile which will write periodically on amazon S3 on the preffix profile
        response= client_firehose.put_record(DeliveryStreamName='projectx-profile',Record={'Data': alias_user_json})
        return(str(response['ResponseMetadata']['HTTPStatusCode']))
    return('Error: Wrong Json Format/fiels')




if __name__ == "__main__":
    app.run(host='0.0.0.0',port=80)

Overwriting main.py


In [8]:
confirmation = 'true'
('post=' + confirmation)

'post=true'

### Confirmations Ifs and sample requests

In [23]:
user_events = {"userId": "fakeid01","events": [{"eventName": "test","metadata": {"test":1},"timestampUTC": 0}]}
alias_user=  {  "newUserId": "string","originalUserId": "string","timestampUTC": 0}
user_profile= {   "userId": "string",  "attributes": {},  "timestampUTC": 0 }

In [17]:
list(user_events.keys())==['userId', 'events']

True

In [18]:
list(alias_user.keys())==  ["newUserId","originalUserId","timestampUTC"]

True

In [24]:
list(user_profile.keys()) ==["userId","attributes","timestampUTC"]

True

# Tutorial Starting working with aws with python
## BOTO3 configuration

In [5]:
import boto3

## create a S3 resource
This will allow us to manage S3 service

In [7]:
s3 = boto3.resource('s3')

## lets see how many buckets/data lakes we have

In [15]:
for bucket in s3.buckets.all():
    print(bucket.name)

projectxs3


# Now lets send the instructions to do a good "açorda alentejana"

In [16]:
data = open('receita_acorda.txt', 'rb')
s3.Bucket('projectxs3').put_object(Key='receita_acorda_', Body=data)

s3.Object(bucket_name='projectxs3', key='receita_acorda_')

so we can acess direct to our s3 bucket but we want the data to pass throught kinesis firehose so...

# KINESISSSS

In [6]:
client = boto3.client('firehose',region_name='us-east-2')

In [7]:
client.list_delivery_streams()

{'DeliveryStreamNames': ['projectx-track', 'projectx_del'],
 'HasMoreDeliveryStreams': False,
 'ResponseMetadata': {'RequestId': 'f83ceecc-0a67-c723-ae58-a32410e2eb82',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'f83ceecc-0a67-c723-ae58-a32410e2eb82',
   'x-amz-id-2': 'L9UFhC6PuyUTCqodeoDB59/xc/uUww/+bZJgnC+cpmZf5AW+VHFJCLo63uJtZwX7MwML/8gYggfKM5t/W2rnmSWxoPFLFzSc',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '88',
   'date': 'Fri, 06 Dec 2019 12:38:00 GMT'},
  'RetryAttempts': 0}}

In [12]:
import json

In [13]:
variables = {"userId": "fakeid05","events": [{"eventName": "test","metadata": {"test":1},"timestampUTC": 0}]}
s=json.dumps(variables)

In [14]:
s

'{"userId": "fakeid05", "events": [{"eventName": "test", "metadata": {"test": 1}, "timestampUTC": 0}]}'

In [15]:
response = client.put_record(
    DeliveryStreamName='projectx_del',
    Record={
        'Data': s
    }
)

In [30]:
type(response['ResponseMetadata']['HTTPStatusCode'])

int

In [5]:
client.describe_delivery_stream(DeliveryStreamName='projectx_del')

{'DeliveryStreamDescription': {'DeliveryStreamName': 'projectx_del',
  'DeliveryStreamARN': 'arn:aws:firehose:us-east-2:841513907133:deliverystream/projectx_del',
  'DeliveryStreamStatus': 'ACTIVE',
  'DeliveryStreamEncryptionConfiguration': {'Status': 'DISABLED'},
  'DeliveryStreamType': 'DirectPut',
  'VersionId': '2',
  'CreateTimestamp': datetime.datetime(2019, 12, 5, 16, 42, 28, 329000, tzinfo=tzlocal()),
  'LastUpdateTimestamp': datetime.datetime(2019, 12, 5, 19, 3, 18, 217000, tzinfo=tzlocal()),
  'Destinations': [{'DestinationId': 'destinationId-000000000001',
    'S3DestinationDescription': {'RoleARN': 'arn:aws:iam::841513907133:role/firehose_delivery_role',
     'BucketARN': 'arn:aws:s3:::projectxs3',
     'Prefix': '',
     'ErrorOutputPrefix': '',
     'BufferingHints': {'SizeInMBs': 5, 'IntervalInSeconds': 300},
     'CompressionFormat': 'UNCOMPRESSED',
     'EncryptionConfiguration': {'NoEncryptionConfig': 'NoEncryption'},
     'CloudWatchLoggingOptions': {'Enabled': True

In [21]:
with open('receita_acorda.txt', 'r') as f:
    for line in f:
        # Read a record of test data
        

            # Put the record into the Firehose stream
        try:
            client.put_record(DeliveryStreamName='projectx_del',
                                           Record={'Data': line})
        except:
            print('error')
        # Put 200 records in a batch
with open('receita_acorda.txt', 'r') as f:
    batch = [{'Data': line} for line in f]  # Read 200 records

In [22]:
batch

[{'Data': '\n'},
 {'Data': '\n'},
 {'Data': 'Como Fazer uma Açorda Alentejana\n'},
 {'Data': '\n'},
 {'Data': 'Pisam-se num almofariz, reduzindo-os a papa, os coentros com os dentes de alho, a que se retirou o grelo, e o sal grosso.\n'},
 {'Data': 'Deita-se esta papa na terrina ou numa tigela de meia cozinha, que neste caso fará ofícios de terrina.\n'},
 {'Data': 'Rega-se com o azeite e escalda-se com a água a ferver, onde previamente se escalfaram os ovos.\n'},
 {'Data': 'Mexe-se a açorda com uma fatia de pão grande, com que se prova a sopa.\n'},
 {'Data': 'A esta sopa dá-se o nome de sopa «azeiteira» ou «sopa mestra».\n'},
 {'Data': 'Introduz-se então no caldo o pão, que foi ou não cortado em fatias ou em cubos com uma faca, ou partido à mão, conforme o gosto.\n'},
 {'Data': 'Depois, tapa-se ou não a açorda, pois uns gostam dela mole e outros apreciam as suas sopas duras.\n'},
 {'Data': 'Os ovos são colocados no prato ou sobre as sopas na terrina, também conforme o gosto.'}]

In [24]:
result = client.put_record_batch(DeliveryStreamName='projectx_del',
                                                      Records=batch)